### **필요한 라이브러리 설치**

**nltk** 
- 한글 문장 단위 분할 함수인 sent_tokenize 지원

**konlpy**
- 한글 형태소 분석기 Okt 지원

In [16]:
!pip install nltk

In [82]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 66.6MB/s 
     |████████████████████████████████| 460kB 45.3MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [80]:
import re
import nltk
import os
from nltk.tokenize import sent_tokenize
from shutil import rmtree

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

구글 드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## **Read Article**

기사 수집 시, 각 언론사 별로 디렉토리를 구성하여 분류하였다.

In [113]:
BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/Text-preprocessing-Data/"
ARTICLE_MEDIA_PATH = os.path.join(BASE_DIR,"articles")

media_list = os.listdir(ARTICLE_MEDIA_PATH)
media_list

['뉴스1',
 'MBN',
 '경향신문',
 'MBC',
 '노컷뉴스',
 'KBS',
 '국민일보',
 'YTN',
 'TV조선',
 'SBS',
 '뉴시스',
 '세계일보',
 '서울신문',
 '연합뉴스',
 '스포츠서울',
 '동아일보',
 '더팩트',
 '연합뉴스TV',
 '디지털타임스',
 '서울경제',
 '중앙일보',
 '오마이뉴스',
 '이데일리',
 '조선일보',
 '파이낸셜뉴스',
 '조선비즈',
 '한겨레']

텍스트 파일에는 기사 제목과 본문, 언론사 이름이 작성되어있다.

In [114]:
for media in media_list:
  media_path = os.path.join(ARTICLE_MEDIA_PATH, media)
  article_list= os.listdir(media_path)
  
  for article in article_list :
    f = open(os.path.join(media_path, article), 'r')
    
    title = f.readline()[:-1]
    content = f.readline()[:-1]
    media = f.readline()[:-1]
    
    f.close()

    print(title)
    print(content)
    print(media)

충북 경찰 4년 연속 총경 2명 배출
(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다. 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다. 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다. 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다. 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.reason@news1.kr© 뉴스1코리아(), 무단 전재 및 재배포 금지
뉴스1
장흥군, 코로나19 예방 위한 "자치경찰 TF팀" 시범운영
(장흥=뉴스1) 박진규 기자 = 전남 장흥군이 코로나19 확산 차단을 위해 자치경찰 TF팀을 구성, 방역수칙 준수에 대한 점검에 나선다.7일 군에 따르면 사회적거리두기 2단계 기간인 17일까지 군 산하 직원들로 구성된 코로나19 '자치경찰 TF팀'을 시범 운영중이다.이들은 장례식당, 식당, 교회, 터미널 등 코로나19 확산 위험도가 높은 다중이용 시설을 대상으로 방역수칙 지키기 캠페인과 점검을 실시하고 있다.자치경찰 TF팀은 올해 7월부터 전국적으로 시행 예정인 '자치경찰제' 도입에 앞서 시행 초기의 혼란과 시행착오를 줄이고, 코로나19 예방 활동을 강화하기 위해 시범적으로 운영되고 있다. 이와 함께 장흥군은 주거밀집 지역에 발판소독매트 및 군 청사 외부에 이동 민원인 상담실 설치, 소독약 무료 배부, 대학생 호루라기 방역단 운영 등 빈틈없는 방역 활동을 추진해 왔다.장흥군은 현재까지 지역 내 감염자가 없는 '코로나19 청정지역'으로 인정받고 있다. 정종순 군수는 "코로나19 장기화로 온 군민이 힘들고 지쳐 있는 상황에서 군민의 안전을 최우선으로 지키기 위해 '자치경찰 TF팀' 운영 등 혼신의 노력을 

파이낸션 언론사 첫번째 기사를 예시로 텍스트 전처리를 수행한다.

In [115]:
media = media_list[-3]
media_path = os.path.join(ARTICLE_MEDIA_PATH, media)
article_list= os.listdir(media_path)

article = article_list[0]
f = open(os.path.join(media_path, article), 'r')
    
title = f.readline()[:-1]
content = f.readline()[:-1]
media = f.readline()[:-1]
    
f.close()

print(title)
print(content)
print(media) 

경찰, 동거 중인 60대 여성 살해한 50대男 체포
[파이낸셜뉴스]    동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다.   서울 서대문경찰서는 살인 혐의로 이모씨(53)를 체포해 조사 중이라고 7일 밝혔다.   이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다.   그는 범행 이후 경찰에 신고해 자수했고, 출동한 경찰관은 오후 5시께 현행범으로 체포했다.   경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다.         ※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지
파이낸셜뉴스


정규식을 사용하여 중복된 스페이스, 탭, 개행을 제거한다.

In [116]:
content = re.sub('\s+', ' ', content)
print(content)

[파이낸셜뉴스] 동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다. 서울 서대문경찰서는 살인 혐의로 이모씨(53)를 체포해 조사 중이라고 7일 밝혔다. 이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다. 그는 범행 이후 경찰에 신고해 자수했고, 출동한 경찰관은 오후 5시께 현행범으로 체포했다. 경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다. ※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지


기사 본문 내용 중 '이모씨(56)'에서 (56)과 같은 나이 표시는 불필요하다 판단하였다.  
이를 정규식으로 제거한다.

In [117]:
content = re.sub(r'\([^)]*\)', '', content)
print(content)

[파이낸셜뉴스] 동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다. 서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다. 이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다. 그는 범행 이후 경찰에 신고해 자수했고, 출동한 경찰관은 오후 5시께 현행범으로 체포했다. 경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다. ※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지


언론사 표시도 불필요하다 판단되어 제거하였다.

In [118]:
content = content.replace(media, '')
print(content)

[] 동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다. 서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다. 이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다. 그는 범행 이후 경찰에 신고해 자수했고, 출동한 경찰관은 오후 5시께 현행범으로 체포했다. 경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다. ※ 저작권자 ⓒ . 무단 전재-재배포 금지


sent_tokenize를 통해 문장 단위로 분할한다.

In [119]:
#문장 단위로 리스트 생성
sentences = sent_tokenize(content)
print(sentences)

['[] 동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다.', '서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다.', '이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다.', '그는 범행 이후 경찰에 신고해 자수했고, 출동한 경찰관은 오후 5시께 현행범으로 체포했다.', '경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다.', '※ 저작권자 ⓒ .', '무단 전재-재배포 금지']


정규식을 통해 텍스트 내에 포함된 특수 문자를 제거한다.

In [120]:
from nltk.tokenize import RegexpTokenizer

retokenize = RegexpTokenizer("[\w]+") 
sentences = [' '.join(retokenize.tokenize(sentence)) for sentence in sentences] 

for sentence in sentences :
  print(sentence)

동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다
서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다
이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다
그는 범행 이후 경찰에 신고해 자수했고 출동한 경찰관은 오후 5시께 현행범으로 체포했다
경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다
저작권자
무단 전재 재배포 금지


본문 내용을 Okt를 통해 토큰화한 후 이어붙여 특수문자를 제거한다.



In [121]:
from konlpy.tag import Okt

okt = Okt()

print(content)
cleanContent = ' '.join(retokenize.tokenize(content))
print(cleanContent) 

[] 동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다. 서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다. 이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다. 그는 범행 이후 경찰에 신고해 자수했고, 출동한 경찰관은 오후 5시께 현행범으로 체포했다. 경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다. ※ 저작권자 ⓒ . 무단 전재-재배포 금지
동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다 서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다 이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다 그는 범행 이후 경찰에 신고해 자수했고 출동한 경찰관은 오후 5시께 현행범으로 체포했다 경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다 저작권자 무단 전재 재배포 금지


NLTK에서 한글 불용어는 제공해주지 않기 때문에 직접 불용어 리스트를 구성해야한다.  
아래 사이트에서 제공하는 한글 불용어 리스트를 사용하였다.  

출처 : https://www.ranks.nl/stopwords/korean

In [109]:
SWORDS_FILE_PATH = os.path.join(BASE_DIR, "StopWordList.txt")
f = open(SWORDS_FILE_PATH, 'r')
swords = f.readlines()
f.close()

swords = [sword.strip() for sword in swords]
swords

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

nouns는 불용어를 제거한 결과이며, sentences는 불용어를 제거하지 않은 리스트다.

In [110]:
nouns = []
for sentence in sentences: 
  if sentence is not '': 
    #불용어 리스트에 포함되지 않는지 체크 
    nouns.append(' '.join([word for word in okt.morphs(sentence) if word not in swords and len(word) >1]))

nouns

['동거 이던 여성 흉기 살해 혐의 받는 50 남성 경찰 붙잡혔다',
 '서울 서대문 경찰서 살인 혐의 이모 체포 조사 중이 라고 7일 밝혔다',
 '이씨 전날 6일 오후 4시 30분 서대문구 홍은동 빌라 60 동거 흉기 찔러 살해 혐의 받는다',
 '범행 이후 경찰 신고 자수 했고 출동 경찰관 오후 5시 현행범 체포 했다',
 '경찰 범행 동기 조사 있으며 구속영장 신청 예정 이다',
 '저작권',
 '무단 배포 금지']

In [111]:
sentences

['동거 중이던 여성을 흉기로 살해한 혐의를 받는 50대 남성이 경찰에 붙잡혔다',
 '서울 서대문경찰서는 살인 혐의로 이모씨를 체포해 조사 중이라고 7일 밝혔다',
 '이씨는 전날인 6일 오후 4시30분께 자신의 집인 서대문구 홍은동의 한 빌라에서 60대 동거녀 A씨를 흉기로 찔러 살해한 혐의를 받는다',
 '그는 범행 이후 경찰에 신고해 자수했고 출동한 경찰관은 오후 5시께 현행범으로 체포했다',
 '경찰은 범행동기를 조사 중에 있으며 구속영장을 신청할 예정이다',
 '저작권자',
 '무단 전재 재배포 금지']